In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

if "JAVA_HOME" not in os.environ:
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64/"


config = {
    "master": "local[7]"
}


spark = SparkSession.builder.appName("example")

for p in config:
    spark = spark.config(p, config[p])

spark = spark.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/02/01 13:30:41 WARN Utils: Your hostname, DESKTOP-FGNDTTH resolves to a loopback address: 127.0.1.1; using 172.30.187.250 instead (on interface eth0)
25/02/01 13:30:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/01 13:30:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
spark\
.range(100000000000000000)

DataFrame[id: bigint]

In [7]:
spark\
.range(1000000)\
.toPandas()[:10]

,id
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [9]:
spark\
.range(5)\
.withColumn("part_id", spark_partition_id())\
.toPandas()

,id,part_id
0,0,1
1,1,3
2,2,4
3,3,6
4,4,7


In [10]:
spark\
.range(5)\
.withColumn("part_id", spark_partition_id())\
.drop("part_id")\
.toPandas()

,id
0,0
1,1
2,2
3,3
4,4


In [16]:
col("a")

Column<'a'>

In [18]:
df = spark\
.range(5)\
.toDF("a")\
.withColumn("is_odd", (col("a") % 2).cast("boolean"))\

pdf = df.toPandas()
pdf

,a,is_odd
0,0,False
1,1,True
2,2,False
3,3,True
4,4,False


In [ ]:
df.limit(3)

In [16]:
df.limit(3).toPandas()

,a,is_odd
0,2,False


In [20]:
df\
.write

In [39]:
df.printSchema()

root
 |-- a: long (nullable = false)
 |-- is_odd: boolean (nullable = true)



In [25]:
df\
.write\
.format("csv")\
.option("header", True)\
.mode("overwrite")\
.save("file:///home/x/odd_even")
#.save("hdfs://localhost:8020/home/x/odd_even")

In [26]:
df\
.write\
.mode("overwrite")\
.save("odd_even_parquet")

In [22]:
df\
.repartition(1)\
.write\
.format("csv")\
.option("header", True)\
.mode("overwrite")\
.save("odd_even")

In [20]:
spark\
.read\
.option("header", True)\
.csv("odd_even")\
.where("is_odd = true")\
.toPandas()

,a,is_odd
0,1,true
1,3,true


In [29]:
spark.read.parquet("odd_even_parquet").toPandas()

,a,is_odd
0,1,True
1,3,True
2,4,False
3,0,False
4,2,False


In [43]:
spark.read.parquet("odd_even_parquet").printSchema()

root
 |-- a: long (nullable = true)
 |-- is_odd: boolean (nullable = true)



In [31]:
df\
.write\
.format("csv")\
.option("header", True)\
.mode("overwrite")\
.partitionBy("is_odd")\
.save("odd_even_partitioned")

In [35]:
spark.read\
.option("header", "true")\
.csv("odd_even")\
.where("is_odd = true")\
.toPandas()

,a,is_odd
0,1,true
1,3,true


In [34]:
spark.read\
.option("header", "true")\
.csv("odd_even_partitioned")\
.toPandas()

,a,is_odd
0,3,true
1,1,true
2,0,false
3,4,false
4,2,false


In [37]:
spark.read\
.option("header", "true")\
.csv("odd_even_partitioned")\
.where("is_odd = false")\
.toPandas()

,a,is_odd
0,0,false
1,4,false
2,2,false


In [38]:
spark.read\
.option("header", "true")\
.csv("odd_even_partitioned")\
.printSchema()

root
 |-- a: string (nullable = true)
 |-- is_odd: string (nullable = true)



In [40]:
spark.read\
.option("header", "true")\
.option("inferSchema", "true")\
.csv("odd_even_partitioned")\
.printSchema()

root
 |-- a: integer (nullable = true)
 |-- is_odd: string (nullable = true)



In [42]:
spark.read\
.option("header", "true")\
.csv("odd_even_partitioned")\
.selectExpr("cast(a as bigint)", "cast(is_odd as boolean)")\
.printSchema()

root
 |-- a: long (nullable = true)
 |-- is_odd: boolean (nullable = true)



In [ ]:
df\
.write\
.format("csv")\
.option("header", True)\
.mode("overwrite")\
.partitionBy("is_odd")\
.bucketBy(2, "a")\
.save("odd_even_partitioned_bucketed")

AnalysisException: 'save' does not support bucketBy right now.